In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime

# Define initial data
clients = ["Tim", "Sofian"]  # Add more clients as needed
initial_investments = [50, 50]  # Initial amounts invested

data = {
    "Client Name": clients,
    "Initial Investment ($)": initial_investments,
}

df = pd.DataFrame(data)

df["Total Start Trading Account Size"] = df["Initial Investment ($)"].sum()

# Calculate proportion of profits
df["Client Profit Proportion"] = df["Initial Investment ($)"] / df["Total Start Trading Account Size"]

# Define dates for the current week
dates = [datetime.date(2025, 3, 3), datetime.date(2025, 3, 4), datetime.date(2025, 3, 5),
         datetime.date(2025, 3, 6), datetime.date(2025, 3, 7)]  # Example week

# Create a DataFrame to track daily balances where users can input their profits/losses
progress_data = {"Client Name": clients}
for date in dates:
    progress_data[date] = [0] * len(clients)  # Initialize with zeroes (to be edited later)
    progress_data[f"Cumulative {date}"] = df["Initial Investment ($)"].copy()

df_progress = pd.DataFrame(progress_data)

# Track cumulative balance for each client
total_weekly_balance = df["Initial Investment ($)"].copy()

# Allow user to manually update profit/loss values
for client in clients:
    print(f"Enter profit/loss values for {client}:")
    for date in dates:
        while True:
            try:
                value = float(input(f"{date}: "))  # User input for daily profit/loss
                df_progress.loc[df_progress["Client Name"] == client, date] = value
                prev_cumulative = df_progress.loc[df_progress["Client Name"] == client, f"Cumulative {date}"]
                new_cumulative = prev_cumulative + value
                df_progress.loc[df_progress["Client Name"] == client, f"Cumulative {date}"] = new_cumulative
                total_weekly_balance.loc[df["Client Name"] == client] += value
                break
            except ValueError:
                print("Invalid input. Please enter a numeric value.")

# Save the data to an Excel file
file_path = "client_trading_tracker.xlsx"
with pd.ExcelWriter(file_path) as writer:
    df.to_excel(writer, sheet_name="Client Summary", index=False)
    df_progress.to_excel(writer, sheet_name="Daily Progress", index=False)

# Visualization of profit progress
for date in dates:
    fig, ax1 = plt.subplots(figsize=(8, 7.5))  # Increase height by 1.5x the max value
    daily_data = df_progress[["Client Name", date, f"Cumulative {date}"]].copy()
    colors = ['green' if profit > 0 else 'red' if profit < 0 else 'gray' for profit in daily_data[date]]
    
    # Adjust bar width to accommodate text visibility
    bar_width = 0.5
    bars = ax1.bar(daily_data["Client Name"], daily_data[date], color=colors, alpha=0.7, label="Daily Profit/Loss", width=bar_width)
    ax1.set_ylabel("Profit/Loss ($)")
    ax1.set_xlabel("Client Name")
    ax1.set_title(f"Client Profit/Loss & Balance on {date}")
    ax1.set_ylim([min(daily_data[date]) * 1.5, max(daily_data[date]) * 1.5])  # Increase height by 1.5x
    ax1.grid(True)
    
    # Annotate daily profit/loss and cumulative balance on bars
    for bar, profit, balance in zip(bars, daily_data[date], daily_data[f"Cumulative {date}"]):
        text_color = 'white' if abs(profit) > 5 else 'black'
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height()/2, f'£{profit:.2f}', 
                 ha='center', va='center', fontsize=10, fontweight='bold', color=text_color, bbox=dict(facecolor='black', edgecolor='white', boxstyle='round,pad=0.3'))
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, f'Total: £{balance:.2f}', 
                 ha='center', va='bottom', fontsize=10, fontweight='bold', color='black')
    
    # Handle zero profit/loss days - Centered Annotation
    if (daily_data[date] == 0).all():  # If all values are zero
        ax1.text(len(clients) / 2 - 0.5, max(ax1.get_ylim()) / 2, "⚖️ Breakeven Day", 
                 ha='center', va='center', fontsize=12, fontweight='bold', color='gray', bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5'))
    
    # Update legend to include total balance for the week
    legend_labels = [f"Daily Profit/Loss {name}" for name in daily_data["Client Name"]]
    total_balance_labels = [f"{name} Total Balance: £{balance:.2f}" for name, balance in zip(daily_data["Client Name"], total_weekly_balance)]
    plt.legend(handles=[bars], labels=legend_labels + total_balance_labels, loc='upper left')
    
    plt.xticks(rotation=45)
    plt.show()

# Display final total balance table
final_balance_df = pd.DataFrame({"Client Name": clients, "Final Total Balance (£)": total_weekly_balance.values})
print("\nFinal Client Balances:")
print(final_balance_df.to_string(index=False))

print(f"Excel file '{file_path}' has been created successfully. You can now edit and track client profit progress for the week.")